# Analysis

In [1]:
import analysis.core as wdt
%matplotlib inline
!nosetests -v

/home/josh/lib/anaconda2/lib/python2.7/site-packages/pyne/serpent.py:11: QAWarning: pyne.serpent is not yet QA compliant.
  warn(__name__ + " is not yet QA compliant.", QAWarning)


/home/josh/lib/anaconda2/lib/python2.7/site-packages/pyne/serpent.py:11: QAWarning: pyne.serpent is not yet QA compliant.
  warn(__name__ + " is not yet QA compliant.", QAWarning)
analysis_tests.TestClass.test_data_frames_mean ... ok
analysis_tests.TestClass.test_data_frames_mean_rel ... ok
analysis_tests.TestClass.test_data_frames_stdev ... ok
analysis_tests.TestClass.test_data_frames_stdev_rel ... ok
analysis_tests.TestClass.test_histogram ... ok
analysis_tests.TestClass.test_histogram_stdev ... ok

----------------------------------------------------------------------
Ran 6 tests in 2.797s

OK


### Base Directory
Set the abslute location of the `wdt_runs` repo.

Ex: if the absolute path of the repo is `/home/josh/repos/wdt_runs/`:

```base_dir = '/home/josh/repos/'```

In [ ]:
base_dir = '/home/josh/repos/'

In [ ]:
#st_vals = [0.1,0.075,0.05]
st_vals = [0.1, 0.075]
wdt_vals = [0.1]
my_analyzer = wdt.Analyzer(st_vals, wdt_vals, base_dir)

In [ ]:
my_analyzer.histogram(['INF_FLX','INF_TOT','INF_ABS'],mean=False).as_matrix()

In [ ]:
my_analyzer.dataSets[0].fom_stat('INF_FLX')

In [ ]:
param = 'INF_SCATT0'
get_mean = False
df=my_analyzer.data_frame(param,rel=False,mean=get_mean,style=False).sort_values(by='Ratio', ascending = not get_mean)[:5]
df2 = pd.concat([df.ix[:,0:2], df.ix[:,-1:]], axis=1)
print df2.to_latex(index=False).replace('\n', ' ')

In [ ]:
plt = my_analyzer.plot_cpu(0.1)

In [ ]:
my_analyzer.histogram(['INF_FLX','INF_TOT','INF_ABS','INF_CAPT','INF_FISS','INF_SCATT0','INF_SCATT1'],mean=True)

In [ ]:
my_analyzer.plot('INF_FLX',[0.1],[0.6])
plt.title('Infinite flux FOM by neutron group for ST threshold 0.1, WDT threshold 0.6', fontsize=16)

In [ ]:
my_analyzer.plot('INF_FLX',[0.1],[0.6])
plt.title('Infinite total cross-section FOM by neutron group for ST threshold 0.1, WDT threshold 0.6', fontsize=14)

In [ ]:
my_analyzer.plot('INF_ABS',[0.1],[0.6])
plt.title('Infinite absorption cross-section FOM by neutron group for ST threshold 0.075, WDT threshold 0.3', fontsize=13)

In [ ]:
my_analyzer.plot('INF_CAPT',[0.1],[0.6])
plt.title('Infinite capture cross-section FOM by neutron group for ST threshold 0.075, WDT threshold 0.3', fontsize=14)